In [ ]:
# Libraries
import yfinance as yf
import pandas as pd
import requests as req
from bs4 import BeautifulSoup
import numpy as np
from tabulate import tabulate
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
import math
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
plt.style.use('seaborn-notebook')

def KeyFunction():
    base_url="https://www.x-rates.com/historical/?from=INR&amount=1&date={:s}"
#     base_url="https://www.x-rates.com/historical/?from=INR&amount=1&date=2020-11-19"
#     l=[]
    datevalue=datetime.today()
    datevalue= datevalue.strftime("%Y-%m-%d")
#     print(base_url.format(datevalue))
    resp=req.get(base_url.format(datevalue))
#     print(resp.status_code)

    soup=BeautifulSoup(resp.text,"html.parser")
    usd=float(soup.find("table",{"class":"ratesTable"}).tbody.tr.select("td")[2].text)
    eur=float(soup.find("table",{"class":"ratesTable"}).tbody.select("tr")[1].select("td")[2].text)
    gbp=float(soup.find("table",{"class":"ratesTable"}).tbody.select("tr")[2].select("td")[2].text)
    aud=float(soup.find("table",{"class":"ratesTable"}).tbody.select("tr")[3].select("td")[2].text)
    cad=float(soup.find("table",{"class":"ratesTable"}).tbody.select("tr")[4].select("td")[2].text)
    sgd=float(soup.find("table",{"class":"ratesTable"}).tbody.select("tr")[5].select("td")[2].text)
    jpy=float(soup.find("table",{"class":"ratesTable"}).tbody.select("tr")[8].select("td")[2].text)
    cny=float(soup.find("table",{"class":"ratesTable"}).tbody.select("tr")[9].select("td")[2].text)
    
#     l.append((usd,eur,gbp,aud,cad,sgd,jpy,cny))
    legend=[["key","Currency","Rate"],[0,"INR",1],[1,"USD",usd],[2,"EURO",eur],[3,"POUND",gbp],[4,"AUSTRALIAN DOLLAR",aud],[5,"CANADIAN DOLLAR",cad],[6,"SINGAPORE DOLAR",sgd],[7,"YEN",jpy],[8,"YUAN",cny]]
# l.append((datevalue,datavalue_dollar))
# print(datavalue_dollar)
# print(l)
    pd.options.display.max_rows=999
#     resultdataframe=pd.DataFrame(l,columns=["DOLLAR","EURO","Pound","Austalia","canada","singapore","japan","china"])
#     print(resultdataframe)
    print(tabulate(legend,headers="firstrow",tablefmt='grid'))
    return legend

In [ ]:
# KeyFunction()

In [ ]:
def MainMenu():
    menu=[["Press","To"],["C","Currency Conversion"],["P","Predicting Values"],["E","Exit"]]
    print(tabulate(menu,headers='firstrow',tablefmt='simple'))
    menu_input=input()
    if(menu_input=="C"):
        CurrencyConversion()
        MainMenu()
    if(menu_input=="P"):
        PredictingValues()
        MainMenu()
    if(menu_input=="E"):
        exit()
    
def CurrencyConversion():
    legend=KeyFunction()
    cc=int(input("enter the currency to be converted: "))
    # cc: currency to be converted
    cci=int(input("enter the currency to be converted into: "))
    # cci: currency to be converted into
    n=True
    while(n):
        print("enter the amount of:",legend[cc+1][1], "to be converted")
        amt_cc= float(input())
        # amount of first currency entered

        value= amt_cc*legend[cc+1][2]/legend[cci+1][2]
        print(amt_cc,legend[cc+1][1]," = ", value, legend[cci+1][1])
        print("Enter 1 to continue more conversion")
        print("Enter 2 to swap the currencies")
        print("Enter 3 to convert in different currency")
        print("Enter 4 to Exit to Main Menu")
        sub_input=int(input())
        if(sub_input==1):continue
        elif(sub_input==2):
            temp=cc
            cc=cci
            cci=temp
            continue
        elif(sub_input==3):
            CurrencyConversion()
        else:
            n=False
#where ever currency conversion is called call the main menu after it
  
    

In [ ]:
def PredictingValues():
#     KeyFunction()
    table=[["Press 1","To View Testing Training Model"],["Press 2"," For Forecasting"]]
    print(tabulate(table))
    pv_input=int(input())
    if(pv_input==1):
        TestTrain()
    if(pv_input==2):
        Forecasting()

In [ ]:
def TestTrain():
    KeyFunction()
    print("Enter the currency key for its model view")
    tt= int(input())
    if(tt==1):DF=yf.download("INR=X")
    if(tt==2):DF=yf.download("EURINR=X")
    if(tt==3):DF=yf.download("GBPINR=X")
    if(tt==4):DF=yf.download("AUDINR=X")
    if(tt==5):DF=yf.download("CADINR=X")
    if(tt==6):DF=yf.download("SGDINR=X")
    if(tt==7):DF=yf.download("INRJPY=X")
    if(tt==8):DF=yf.download("CNYINR=X")
    # Only keeping close columns
    print(DF.tail(5))
    Df = DF[['Close']]

    # Droping rows with missing values
    Df = Df.dropna()

    Df['S_3'] = Df['Close'].rolling(window=3).mean()
    Df['S_9'] = Df['Close'].rolling(window=9).mean()
    Df['next_day_price'] = Df['Close'].shift(-1)
    print(Df.tail(5))

    Df = Df.dropna()
    X = Df[['S_3', 'S_9']]

    # Defining dependent variable
    y = Df['next_day_price']

    # Spliting the data into train and test dataset
    t = .8
    t = int(t*len(Df))

    # Train dataset
    X_train = X[:t]
    y_train = y[:t]

    # Test dataset
    X_test = X[t:]
    y_test = y[t:]

    # Creating a linear regression model
    linear = LinearRegression().fit(X_train, y_train)
    print("Linear Regression model")
    print("Price (y) = %.2f * 3 Days Moving Average (x1) \+ %.2f * 9 Days Moving Average (x2) \
    + %.2f (constant)" % (linear.coef_[0], linear.coef_[1], linear.intercept_))

    # Predicting the prices
    predicted_price = linear.predict(X_test)
    predicted_price = pd.DataFrame(
    predicted_price, index=y_test.index, columns=['price'])
    predicted_price.plot(figsize=(10, 7))
    y_test.plot()
    
    

In [ ]:
def Forecasting():
    KeyFunction()
    print("Enter the currency key for its model view")
    tt= int(input())
    if(tt==1):DF=yf.download("INR=X",'2013-9-23', '2020-11-24', auto_adjust=True)
    if(tt==2):DF=yf.download("EURINR=X",'2013-9-23', '2020-11-24', auto_adjust=True)
    if(tt==3):DF=yf.download("GBPINR=X",'2013-9-23', '2020-11-24', auto_adjust=True)
    if(tt==4):DF=yf.download("AUDINR=X",'2013-9-23', '2020-11-24', auto_adjust=True)
    if(tt==5):DF=yf.download("CADINR=X",'2013-9-23', '2020-11-24', auto_adjust=True)
    if(tt==6):DF=yf.download("SGDINR=X",'2013-9-23', '2020-11-24', auto_adjust=True)
    if(tt==7):DF=yf.download("INRJPY=X",'2013-9-23', '2020-11-24', auto_adjust=True)
    if(tt==8):DF=yf.download("CNYINR=X",'2013-9-23', '2020-11-24', auto_adjust=True)
    df = DF[['Open',  'High',  'Low',  'Close','Volume']]
    df['HL_PCT'] = (df['High'] - df['Low']) / df['Close'] * 100.0
    df['PCT_change'] = (df['Close'] - df['Open']) / df['Open'] * 100.0

    df = df[['Close', 'HL_PCT', 'PCT_change', 'Volume']]
    forecast_col = 'Close'
    df.fillna(value=-99999, inplace=True)
    forecast_out = int(math.ceil(0.01 * len(df)))
    df['label'] = df[forecast_col].shift(-forecast_out)

    X = np.array(df.drop(['label'], 1))
    X = preprocessing.scale(X)
    X_lately = X[-forecast_out:]
    X = X[:-forecast_out]

    df.dropna(inplace=True)

    y = np.array(df['label'])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    clf = LinearRegression(n_jobs=-1)
    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test)

    forecast_set = clf.predict(X_lately)
    df['Forecast'] = np.nan

    last_date = df.iloc[-1].name
    last_unix = last_date.timestamp()
    one_day = 86400
    next_unix = last_unix + one_day

    for i in forecast_set:
        next_date = datetime.datetime.fromtimestamp(next_unix)
        next_unix += 86400
        df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)]+[i]

    df['Close'].plot()
    df['Forecast'].plot()
    plt.legend(loc=4)
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.show()

In [ ]:
MainMenu()